In [ ]:
# =====================================================
# FAKE NEWS DETECTION – FIXED FOR RICKSTELLO DATASET
# =====================================================

!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from google.colab import drive

warnings.filterwarnings("ignore")

# 1. SETUP
device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
print(f"Device: {device_name} | CUDA: {torch.cuda.is_available()}")

drive.mount('/content/drive', force_remount=False)
OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_DistilBERT_Fixed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. TẢI DATASET AN TOÀN
print("\n=== ĐANG TẢI DATASET: rickstello/FakeNewsNet ===")
try:
    # Dataset này thường chia thành 2 phần: gossipcop và politifact
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Không tải được config con ({e}). Đang thử tải default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

print(f"Tổng số dòng: {len(df)}")
print(f"Danh sách cột gốc: {list(df.columns)}") # In ra để kiểm tra

# 3. TỰ ĐỘNG DÒ TÌM & MAP CỘT (QUAN TRỌNG)
# Danh sách các tên cột có thể xuất hiện
candidate_text = ['news_content', 'text', 'content', 'body', 'full_text']
candidate_title = ['title', 'news_title', 'headline']
candidate_label = ['real', 'label', 'class', 'fake']

# Tìm tên cột thực tế trong df
text_col = next((c for c in candidate_text if c in df.columns), None)
title_col = next((c for c in candidate_title if c in df.columns), None)
label_col = next((c for c in candidate_label if c in df.columns), None)

print(f"\nMapping cột tìm được: Text='{text_col}' | Title='{title_col}' | Label='{label_col}'")

if not label_col:
    raise ValueError("❌ LỖI: Không tìm thấy cột nhãn (label/real). Hãy kiểm tra lại danh sách cột gốc ở trên.")

# Chuẩn hóa tên cột nhãn về 'label'
df['label'] = df[label_col]

# Logic: Nếu cột tồn tại -> lấy dữ liệu & điền NA. Nếu không -> dùng chuỗi rỗng.
if title_col:
    s_title = df[title_col].fillna('')
else:
    s_title = ""

if text_col:
    s_text = df[text_col].fillna('')
else:
    s_text = ""

# Ghép title + text
df['content'] = s_title + " [SEP] " + s_text

# 4. LÀM SẠCH VÀ CHUẨN HÓA
def clean_text(text):
    if not isinstance(text, str): return ""
    t = text.lower()
    t = re.sub(r'https?://\S+|www\.\S+', ' ', t)
    t = re.sub(r'<.*?>', ' ', t)
    t = re.sub(r'[^a-zA-Z0-9\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

print("Đang làm sạch văn bản...", end="")
df['content'] = df['content'].apply(clean_text)
# Loại bỏ dòng quá ngắn hoặc rỗng
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])
print(f" → Sau xử lý: {len(df):,}")

# --- XỬ LÝ Ý NGHĨA NHÃN (LABEL MAPPING) ---
# Kiểm tra nhanh:
print(f"Phân bố nhãn: {df['label'].value_counts(normalize=True).to_dict()}")

# 5. CHIA DỮ LIỆU
classes = np.unique(df['label'])
# Tính class weight để cân bằng (vì tin thật thường nhiều hơn tin giả trong bộ này)
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {k: float(v) for k, v in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 6. TOKENIZER & MODEL
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(classes))

# Cập nhật Label Mapping 
if label_col == 'real':
    model.config.id2label = {0: "Fake", 1: "Real"}
    model.config.label2id = {"Fake": 0, "Real": 1}
else:
    # Fallback mặc định
    model.config.id2label = {0: "Class 0", 1: "Class 1"}
    model.config.label2id = {"Class 0": 0, "Class 1": 1}

# 7. TRAINER (Weighted Loss)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        weight_tensor = torch.tensor(list(class_weight_dict.values()), dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight_tensor)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("\n=== BẮT ĐẦU HUẤN LUYỆN ===")
trainer.train()

# Đánh giá cuối cùng
print("\nĐÁNH GIÁ TRÊN TẬP TEST:")
results = trainer.evaluate(tokenized["test"])
print(results)

# Lưu model
final_path = os.path.join(OUTPUT_DIR, "final_model_fixed")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"\nĐã lưu model tại: {final_path}")

Device: Tesla T4 | CUDA: True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== ĐANG TẢI DATASET: rickstello/FakeNewsNet ===
⚠️ Không tải được config con (BuilderConfig 'gossipcop' not found. Available: ['default']). Đang thử tải default...
Tổng số dòng: 23196
Danh sách cột gốc: ['title', 'news_url', 'source_domain', 'tweet_num', 'real']

Mapping cột tìm được: Text='None' | Title='title' | Label='real'
Đang làm sạch văn bản... → Sau xử lý: 21,287
Phân bố nhãn: {1: 0.7577394654014187, 0: 0.2422605345985813}
Class weights: {np.int64(0): 2.063893736668606, np.int64(1): 0.6598574085554867}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== BẮT ĐẦU HUẤN LUYỆN ===


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.470600,0.455038,0.803194,0.812097,0.833115,0.803194
2,0.360100,0.521910,0.807421,0.815612,0.834154,0.807421
3,0.257900,0.624965,0.837952,0.838734,0.839613,0.837952
4,0.174200,0.858575,0.840301,0.838646,0.837393,0.840301
5,0.124700,1.009838,0.837013,0.835616,0.834496,0.837013



ĐÁNH GIÁ TRÊN TẬP TEST:


{'eval_loss': 0.600078821182251, 'eval_accuracy': 0.8337247534053547, 'eval_f1': 0.836164061617057, 'eval_precision': 0.839555533687898, 'eval_recall': 0.8337247534053547, 'eval_runtime': 1.975, 'eval_samples_per_second': 1077.988, 'eval_steps_per_second': 67.849, 'epoch': 5.0}

Đã lưu model tại: /content/drive/MyDrive/FakeNewsNet_DistilBERT_Fixed/final_model_fixed
